In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

# Read the data
X = pd.read_csv('D:/code/Data/restaurant-revenue-prediction/train.csv', index_col='Id')
X_test_full = pd.read_csv('D:/code/Data/restaurant-revenue-prediction/test.csv', index_col='Id')





In [3]:
X.head()

,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,P6,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
Id,,,,,,,,,,,,,,,,,,,,,
0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,2,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,3,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [4]:
X.columns

Index(['Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5',
       'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16',
       'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26',
       'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36',
       'P37', 'revenue'],
      dtype='object')

In [6]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['revenue'], inplace=True)
y = X.revenue              
X.drop(['revenue'], axis=1, inplace=True)

In [7]:
# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)





In [18]:
#iterating through the hyperparameters
'''paramaters to iterate:
    n_estimators
    learning_rate
    early_stopping_rounds
    tree_depth
    
    '''



data=[]

for est in range(10000,10001,5000):
    for learn in np.arange(.008,.025,.001):
        for early in range(50):
          
            my_model = XGBRegressor(objective ='reg:squarederror',n_estimators=est, learning_rate=learn, n_jobs=8)
            my_model.fit(X_train, y_train, early_stopping_rounds=early, eval_set=[(X_valid, y_valid)], verbose=False)
            predictions = my_model.predict(X_valid)
            mae=mean_absolute_error(predictions, y_valid)
            
            data.append({'estimators':est,'learning_rate':learn,'early_stopping_rounds':early,'mae':mae})
            print(f'{est} {learn} {early} {mae}')
            
hyper=pd.DataFrame(data)
print(hyper)
hyper_min=hyper[hyper.mae == hyper.mae.min()]
print(hyper_min)


hyper.to_csv(r'D:/code/Data/hyper1.csv',index=False, header =1)
hyper_min.to_csv(r'D:/code/Data/hyper_min1.csv',index=False, header =1)



10000 0.008 0 1780010.2410714286
10000 0.008 1 1780010.2410714286
10000 0.008 2 1780010.2410714286
10000 0.008 3 1716569.5625
10000 0.008 4 1716569.5625
10000 0.008 5 1715627.7857142857
10000 0.008 6 1696519.1875
10000 0.008 7 1694384.705357143
10000 0.008 8 1692375.830357143
10000 0.008 9 1674967.0892857143
10000 0.008 10 1674967.0892857143
10000 0.008 11 1674967.0892857143
10000 0.008 12 1674967.0892857143
10000 0.008 13 1675007.5982142857
10000 0.008 14 1675007.5982142857
10000 0.008 15 1675007.5982142857
10000 0.008 16 1675007.5982142857
10000 0.008 17 1675007.5982142857
10000 0.008 18 1675007.5982142857
10000 0.008 19 1675007.5982142857
10000 0.008 20 1675007.5982142857
10000 0.008 21 1675007.5982142857
10000 0.008 22 1675283.5
10000 0.008 23 1675283.5
10000 0.008 24 1675283.5
10000 0.008 25 1675283.5
10000 0.008 26 1675283.5
10000 0.008 27 1675283.5
10000 0.008 28 1675283.5
10000 0.008 29 1675283.5
10000 0.008 30 1675283.5
10000 0.008 31 1675283.5
10000 0.008 32 1675283.5
10000 0

10000 0.011000000000000003 40 1700571.4017857143
10000 0.011000000000000003 41 1700571.4017857143
10000 0.011000000000000003 42 1700571.4017857143
10000 0.011000000000000003 43 1700571.4017857143
10000 0.011000000000000003 44 1700571.4017857143
10000 0.011000000000000003 45 1700571.4017857143
10000 0.011000000000000003 46 1700571.4017857143
10000 0.011000000000000003 47 1700571.4017857143
10000 0.011000000000000003 48 1700571.4017857143
10000 0.011000000000000003 49 1700571.4017857143
10000 0.012000000000000004 0 1792386.642857143
10000 0.012000000000000004 1 1792386.642857143
10000 0.012000000000000004 2 1783783.4017857143
10000 0.012000000000000004 3 1724674.3214285714
10000 0.012000000000000004 4 1724674.3214285714
10000 0.012000000000000004 5 1727030.6964285714
10000 0.012000000000000004 6 1725970.392857143
10000 0.012000000000000004 7 1690750.0
10000 0.012000000000000004 8 1690750.0
10000 0.012000000000000004 9 1690750.0
10000 0.012000000000000004 10 1690750.0
10000 0.012000000000

10000 0.015000000000000006 21 1685083.294642857
10000 0.015000000000000006 22 1685083.294642857
10000 0.015000000000000006 23 1685083.294642857
10000 0.015000000000000006 24 1685083.294642857
10000 0.015000000000000006 25 1685083.294642857
10000 0.015000000000000006 26 1685083.294642857
10000 0.015000000000000006 27 1685083.294642857
10000 0.015000000000000006 28 1685083.294642857
10000 0.015000000000000006 29 1685083.294642857
10000 0.015000000000000006 30 1685083.294642857
10000 0.015000000000000006 31 1685083.294642857
10000 0.015000000000000006 32 1685083.294642857
10000 0.015000000000000006 33 1685083.294642857
10000 0.015000000000000006 34 1685083.294642857
10000 0.015000000000000006 35 1685083.294642857
10000 0.015000000000000006 36 1685083.294642857
10000 0.015000000000000006 37 1685083.294642857
10000 0.015000000000000006 38 1687184.1875
10000 0.015000000000000006 39 1687184.1875
10000 0.015000000000000006 40 1687184.1875
10000 0.015000000000000006 41 1687184.1875
10000 0.0150

10000 0.01800000000000001 43 1700933.5357142857
10000 0.01800000000000001 44 1700933.5357142857
10000 0.01800000000000001 45 1700933.5357142857
10000 0.01800000000000001 46 1700933.5357142857
10000 0.01800000000000001 47 1700933.5357142857
10000 0.01800000000000001 48 1700933.5357142857
10000 0.01800000000000001 49 1700933.5357142857
10000 0.01900000000000001 0 1789745.125
10000 0.01900000000000001 1 1789745.125
10000 0.01900000000000001 2 1789745.125
10000 0.01900000000000001 3 1718838.767857143
10000 0.01900000000000001 4 1718838.767857143
10000 0.01900000000000001 5 1718301.142857143
10000 0.01900000000000001 6 1718301.142857143
10000 0.01900000000000001 7 1718301.142857143
10000 0.01900000000000001 8 1683870.9464285714
10000 0.01900000000000001 9 1683870.9464285714
10000 0.01900000000000001 10 1683870.9464285714
10000 0.01900000000000001 11 1683870.9464285714
10000 0.01900000000000001 12 1683870.9464285714
10000 0.01900000000000001 13 1683870.9464285714
10000 0.01900000000000001 14

10000 0.022000000000000013 16 1702156.107142857
10000 0.022000000000000013 17 1702156.107142857
10000 0.022000000000000013 18 1702156.107142857
10000 0.022000000000000013 19 1702156.107142857
10000 0.022000000000000013 20 1702156.107142857
10000 0.022000000000000013 21 1702156.107142857
10000 0.022000000000000013 22 1702156.107142857
10000 0.022000000000000013 23 1702156.107142857
10000 0.022000000000000013 24 1702156.107142857
10000 0.022000000000000013 25 1702156.107142857
10000 0.022000000000000013 26 1702156.107142857
10000 0.022000000000000013 27 1702156.107142857
10000 0.022000000000000013 28 1702156.107142857
10000 0.022000000000000013 29 1702156.107142857
10000 0.022000000000000013 30 1702156.107142857
10000 0.022000000000000013 31 1702156.107142857
10000 0.022000000000000013 32 1702156.107142857
10000 0.022000000000000013 33 1702156.107142857
10000 0.022000000000000013 34 1702156.107142857
10000 0.022000000000000013 35 1702156.107142857
10000 0.022000000000000013 36 1702156.10

In [19]:
#print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

my_model = XGBRegressor(objective ='reg:squarederror',n_estimators=10000, learning_rate=.021, n_jobs=8)
my_model.fit(X_train, y_train, early_stopping_rounds=8, eval_set=[(X_valid, y_valid)], verbose=False)

preds_test = my_model.predict(X_test)

In [21]:
output = pd.DataFrame({'Id': X_test.index, 'Prediction': preds_test})

output.to_csv(r'D:/code/Data/restaurant-revenue-prediction/revenue_submission_3.csv',index=False, header =1)